In [19]:
from textblob import TextBlob
import json
import numpy as np
import time
import itertools
import re
import string
'''
def load_data(n: int) -> list:
    data = []
    with open("gg2020.json") as file:
        for line in file.readlines():
            data.append(json.loads(line[:-1])["text"])
            if len(data) > n: break

    return data
'''

def load_data(year):
    if year == "2013" or year == "2015"or year == "2018"or year == "2019":
        f = open('gg'+year+'.json')
        data = json.load(f)
        TWEETS = [tweet['text'] for tweet in data]
        #return TWEETS
    else:                                              
        with open('gg'+year+'.json', encoding='utf8') as json_file:
            data = [json.loads(line) for line in json_file]
        TWEETS = [tweet['text'] for tweet in data]
            #return TWEETS
    TWEETS.sort()
    TWEETS = list(TWEETS for TWEETS,_ in itertools.groupby(TWEETS))
    return TWEETS
        
        

def sentiment(sentences: list) -> tuple:
    '''Can be used on entity's sentiment and red carpet, Humor (find the word dress/joke first)
       best dressed (++), worst dressed (-+), most discussed(.-), most controversial(--),'''
    polarity, subjectivity = 0, 0
    for sentence in sentences:
        tb = TextBlob(sentence.replace("\n", ""))
        polarity += tb.sentiment.polarity
        subjectivity += tb.sentiment.subjectivity
    
    polarity, subjectivity = polarity/len(sentences), subjectivity/len(sentences)

    return polarity, subjectivity

def find_keyword(sentences: list, keyword: str) -> list:
    '''keyword should be in lower case'''
    data = []
    for sentence in sentences:
        tb = TextBlob(sentence)
        if keyword in tb.lower(): data.append(sentence)
    
    return data

def additional(entities: list, typ: str, data: list):
    if typ not in ["dress", "joke", "sentiment", "parties", "act"]: raise Exception

    P, S = [], []

    for entity in entities:
        _data = find_keyword(data, entity)

        if typ in ["dress", "joke", "act"]: __data = find_keyword(_data, typ)
        elif typ in ["parties"]:
            # Find other actor
            pass
        
        if not len(__data):
            P.append(0)
            S.append(0.5)
            continue
        
        p, s = sentiment(__data)
        P.append(p)
        S.append(s)

    P, S = np.array(P), np.array(S)
    
    if typ == "dress":
        print("best dressed", entities[np.argmin(np.power(P-1, 2) + np.power(S-1, 2))])
        print("worst dressed", entities[np.argmin(np.power(P+1, 2) + np.power(S-1, 2))])
        print("most discussed", entities[np.argmin(S)])
        print("most controversial", entities[np.argmin(np.power(P+1, 2) + np.power(S, 2))])

    elif typ == "joke":
        pass



In [91]:
def find_party(tweet):
    ss = dict()
    party_pattern = re.compile(r'at [a-zA-Z\s]+(?=\sparty)')
    for item in tweet:
        pp = re.findall(party_pattern, item)
        for party in pp:
            if party[3:] not in ss:
                ss[party[3:]] = 1
            else:
                ss[party[3:]] += 1
    
    ordered_p_tweet = sorted(ss.items(), key=lambda x: x[1], reverse=True)
    stop_party = ['after', 'window']
    clean_p_tweet = []
    for item in ordered_p_tweet:
        flag = True
        for word in stop_party:
            if word in item[0].lower().split():
                flag = False
                break
        if flag:
            clean_p_tweet.append(item)
                
    nonsense = ['a', 'the', 'an', 'his', 'her', 'their', 'my', 'a golden globe', 'the golden globes','our golden globes', 'golden globe', 'golden globes', 'your golden globes']
    _clean_p_tweet = []
    for item in clean_p_tweet:
        flag = True
        for w in nonsense:
            if w == item[0].lower():
                flag = False
                break
        if flag:
            _clean_p_tweet.append(item)
        
    
    return _clean_p_tweet

data = load_data("2020")
#clean_data(data)
p_tweet = find_party(data)
print (len(p_tweet))

65


In [92]:
print(p_tweet)

[('the YSL pre Golden Globes', 6), ('the pre Golden Globes', 4), ('a Netflix Golden Globes', 4), ('the same', 2), ('the InStyle Golden Globes', 2), ('Audrina Patridge stepped out in and more', 2), ('Screen Slate Golden Globe viewing', 1), ('a Hollywood', 1), ('the YSL Golden Globe', 1), ('walks into the', 1), ('a golden globe watch', 1), ('the Democrat', 1), ('the golden globes or at a golden globes', 1), ('for the beach', 1), ('the next Golden Globe', 1), ('this Golden Globes', 1), ('a golden globes pre', 1), ('a Golden Globes viewing', 1), ('the Golden Globes is dressed like someone at a', 1), ('our Golden Globes watch', 1), ('the wmag', 1), ('HBO', 1), ('brain is throwing a', 1), ('the wme Golden Globes', 1), ('time at the HBO Golden Globes', 1), ('lionsgate Golden Globes', 1), ('their respective football', 1), ('this Golden Globes viewing', 1), ('all these people wearing velvet', 1), ('a Golden Globes watch', 1), ('BAFTA', 1), ('the Golden Globes watch', 1), ('this family', 1), ('a

In [93]:
data = load_data("2013")
#clean_data(data)
p_tweet = find_party(data)
print (len(p_tweet))

19


In [94]:
p_tweet

[('the InStyle', 7),
 ('this', 3),
 ('that', 2),
 ('the vicious queens at this', 1),
 ('W Magazine pre Golden Globes', 1),
 ('whole foods for golden globes', 1),
 ('time at the Golden Globes', 1),
 ('the Beverly Hilton is basically a Hallowhedon', 1),
 ('my all star dinner', 1),
 ('my birthday', 1),
 ('other girl she met at that', 1),
 ('her globes watch', 1),
 ('there is no funner dinner', 1),
 ('Twitter', 1),
 ('dinner', 1),
 ('a track pasta', 1),
 ('Bill is in the place its a', 1),
 ('a golden globes', 1),
 ('a birthday', 1)]

In [95]:
data = load_data("2015")
#clean_data(data)
p_tweet = find_party(data)
print (len(p_tweet))

152


In [96]:
p_tweet

[('the HBO Golden Globes', 18),
 ('a Golden Globes', 11),
 ('this', 7),
 ('the GG', 7),
 ('the HBO', 7),
 ('Instyle', 5),
 ('the InStyle Golden Globes', 5),
 ('every', 4),
 ('a dinner', 4),
 ('the INSTYLE', 4),
 ('a golden globes', 3),
 ('Fox', 3),
 ('this Golden Globes', 3),
 ('my GG', 3),
 ('a pre Golden Globes', 3),
 ('the CAA Golden Globes', 3),
 ('the Pre Golden Globes', 2),
 ('the Fox Golden Globes', 2),
 ('the golden globe', 2),
 ('GG watch', 2),
 ('the InStyle viewing', 2),
 ('as you leave his', 2),
 ('bella will attend the golden globes', 2),
 ('the fox', 2),
 ('the NBC viewing', 2),
 ('this viewing', 2),
 ('writer and great actor at the golden globe', 2),
 ('our golden globe', 2),
 ('the Hbo', 2),
 ('mocking North Korea tonight is a', 2),
 ('NBC viewing', 2),
 ('the Golden Globes Instyle', 2),
 ('this golden globes', 2),
 ('the Golden Globes I am following Bill Murray to whatever', 2),
 ('nice couple at a', 2),
 ('the lovely view that the Fox', 2),
 ('the InStyle watch', 2),


In [107]:
data = load_data("2020")
#data = clean_data(data)
p_tweet = find_party(data)
print (len(p_tweet))

most_party_twitter = []
for item in data:
    if p_tweet[0][0] in item:
        most_party_twitter.append(item)
print ("Party People Most Talk:", p_tweet[0][0])
print ("Sentiment Score for the Party:", sentiment(most_party_twitter))


65
Party People Most Talk: the YSL pre Golden Globes
Sentiment Score for the Party: (0.3, 0.5)


In [109]:
data = load_data("2013")
#data = clean_data(data)
p_tweet = find_party(data)
print (len(p_tweet))

most_party_twitter = []
for item in data:
    if p_tweet[0][0] in item:
        most_party_twitter.append(item)
print ("Party People Most Talk:", p_tweet[0][0])
print ("Sentiment Score for the Party:", sentiment(most_party_twitter))

19
Party People Most Talk: the InStyle
Sentiment Score for the Party: (0.3263392857142858, 0.5642857142857143)


In [110]:
data = load_data("2015")
#data = clean_data(data)
p_tweet = find_party(data)
print (len(p_tweet))

most_party_twitter = []
for item in data:
    if p_tweet[0][0] in item:
        most_party_twitter.append(item)
print ("Party People Most Talk:", p_tweet[0][0])
print ("Sentiment Score for the Party:", sentiment(most_party_twitter))

152
Party People Most Talk: the HBO Golden Globes
Sentiment Score for the Party: (0.20460526315789462, 0.5837719298245613)
